In [ ]:
#hide

# RaDICaL Dataset SDK

> The RaDICaL Dataset: A synchronized and calibrated low-level Radar, RGB-D and IMU dataset.

This is pre-alpha research quality code. Bug reports are very much appreciated.

## Install

`pip install radical-dataset`

## How to use